In this task I will have a couple of tasks to performe in my process of sentiment analysis:
## Sentiment Analysis process:
    1.  Analyze the Data and understand how I should handle it.
    2.  Load the necessary data and filter out redundant information.
    3.  Handle the data and prepare it such that it fits the models I will be using. This step will contain a couple of subprocess:
        a. Tokenizing the data.
        b. Sanitizing the data by removing all the stopwords, punctuation and numbers that will only reduce the performance of the Sentiment Analysis model. 
        c. Lemmatizing the data in order to make it more understandable for the model.
        d. Vectorizing in order to fit the data to an graph that I will be using when performing the real Sentiment Analysis. 
    4. Finding the most optimal algorithm.
        a. I will have to try different algorithms with cross validation and use their results in order to find which one fits the data the best. 
        b. In addition I will need to find the most optimal parameters. I will see later how I choose to search for them. 
    5. Training the model with the preprocessed data.
    6. Testing the model with the preprocessed data. 

And then I will be evaluating the data, to see if my score is sufficient or not. 

In [1]:
import json
import nltk
import spacy
import numpy as np
import requests


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# This is for the sentimental analysis more exactly for comparing the different algorithms with eachother. 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

!pip install https://github.com/explosion/spacy-models/releases/download/nb_core_news_sm-3.1.0/nb_core_news_sm-3.1.0.tar.gz

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
      --------------------------------------- 0.2/16.1 MB 4.8 MB/s eta 0:00:04
     -- ------------------------------------- 1.0/16.1 MB 10.2 MB/s eta 0:00:02
     ----- ---------------------------------- 2.2/16.1 MB 15.4 MB/s eta 0:00:01
     -------- ------------------------------- 3.4/16.1 MB 18.1 MB/s eta 0:00:01
     ------------ --------------------------- 5.1/16.1 MB 21.6 MB/s eta 0:00:01
     --------------- ------------------------ 6.3/16.1 MB 23.6 MB/s eta 0:00:01
     ----------------- ---------------------- 7.2/16.1 MB 22.9 MB/s eta 0:00:01
     -------------------- ------------------- 8.4/16.1 MB 22.4 MB/s eta 0:00:01
     ------------------------ --------------- 9.8/16.1 MB 24.1 MB/s eta 0:00:01
     --------------------------- ----------- 11.3/16.1 MB 27.3 MB/s eta 0:00:01
     ------------------------------- ------- 12.9/1

### 1.Analyzing the data & understanding its structure

In [2]:
def data_analysys(json_data) -> list:
    keys_in_data = set()
    print("The size of the data is:", len(json_data))

    for item in json_data:
        keys = item.keys()
        keys_in_data.update(keys)

    print(f'Main keys of this data are: {keys_in_data}')

I will use the function above to analyze the structure of the content of this file. To learn how I should be preprocessing it. And how I can make it more efficient.

In the cell below I simply load the documents from the url's and print the recieved data using the previously defined data_analysys() function.

In [3]:
# Replace this URL with the raw URL of the file you want to fetch from GitHub
train_url = "https://raw.githubusercontent.com/ltgoslo/norec_sentence/main/binary/train.json"
test_url = "https://raw.githubusercontent.com/ltgoslo/norec_sentence/main/binary/test.json"

# Fetch the content of the file
train_response = requests.get(train_url)
test_response = requests.get(test_url)

# Raise an exception if there was an error fetching the file
train_response.raise_for_status() 
test_response.raise_for_status()

# Get the content of the file as a string
json_training_data = json.loads(train_response.text)
json_test_data = json.loads(test_response.text)

# This is just for analytic reasons. Not for loading the content:
data_analysys(json_training_data)
data_analysys(json_test_data)

The size of the data is: 3894
Main keys of this data are: {'text', 'sent_id', 'label'}
The size of the data is: 583
Main keys of this data are: {'text', 'sent_id', 'label'}


### 2.Loading the data & filtering out what's redundant

As you can see there's an extra, redundant label that I do not have to consider doing my sentiment analysis, and that is the "sent_id" label. The date the message was sent is irrelevant therefore I can remove it from my data to do more efficient model training. 

I will only be using the text and label part of this data so I continue in defining a function that reads only 'label' & 'text' part of the data. 


"data_reading()" extracts the data from the json file provided in a structured way: ["sentance", "sentiment"]. I consider the data not to complex so I decided to not bother with visualizing the data to much.

In [4]:
def data_reading(json_data) -> list:
    # Extract message_data directly using a list comprehension
    message_data = [[category["text"], category["label"]] for category in json_data]

    return message_data

Then I proceed to define the global variable to make sure that I only load this data once. This is done in order to make the processing much faster. 

In [5]:
train_data = data_reading(json_training_data)
test_data = data_reading(json_test_data)

### 3.Handling & Praparing the data for my model.

Before creating the function I was struggling with accessing the data in an efficient way. I wanted to divide: the sanitizing, tokenizing, lemmatizing and vectorizing of the data. But I quickly figured that it would have cost a lot of computation time since I would have to access the same data multiple times in nested for loops. So instead I created preprocessing() which did sanitizing with tokenizing, and then lemmatized the sentances in one go. In addition to that, this function divides the preprocessed data into sentances and sentiments, which are perfectly prepared to be vectorized. 

In [6]:
def preprocessing(data):
    sentences = []
    sentiments = []
    lemmatizer = spacy.load("nb_core_news_sm")  # Norwegian lemmatization model.

    for chunk in data:
        text, sentiment = chunk[0], chunk[1]
        # Here I tokenize the data, and prepare it to run it through a sanitizer.
        pre_lemmatized = lemmatizer(text)


        # Here I add the lemmatized lowercase word to the "lemmatized" list,
        # but only after it passes all the tests, to check if it's a legitimate word.  
        lemmatized = [root_word.lemma_.lower() for root_word in pre_lemmatized
                      if (not root_word.is_punct
                          and not root_word.is_currency
                          and not root_word.is_digit
                          and not root_word.is_space
                          and not root_word.is_stop
                          and not root_word.like_num)]


        # This if statement checks if the list is empty or not.
        # If it is, it simply continues to the next sentance without adding the 
        # sentiment. That provides a reasurance that there will not be any sentiments
        # attached to a empty list.
        if len(lemmatized)==0:
            continue
        else:
            # And here I join each processed word to one sentance in order to do
            # sentance sentimental analysis.
            sentence = ' '.join(lemmatized)
            sentences.append(sentence)
            sentiments.append(sentiment)

    return sentences, sentiments


In the last part of the for loop, where I have that if statement I figured that when filtering out all the stopwords all slang words may be filtered out too. So I needed to add that to not end up with some sentiments being attached to empty lists. 

So I performe the vectorization using CountVectorizer() & LabelEncoder() because that seemed most trivial of every other way that I stumbled upon. I considered if I should use TF-IDF or Bag-of-Words vectorization. But I concluded with TF-IDF seemed to be more fit for bigger datas with variaty of lenghts and inputs. Meanwhile here I have tree prepared documents with the same format which will not vary as much as data from the real world would. 
And my implementation is shown bellow:

In [7]:
def vectornihilation(x_axis, y_axis):
    vectorizer = CountVectorizer()
    labelizer = LabelEncoder()
    X_matrix = vectorizer.fit_transform(x_axis)
    Y_matrix = labelizer.fit_transform(y_axis)

    return X_matrix, Y_matrix, vectorizer, labelizer

And this function gives me a matrix as the x vectors and a numpy array with vector representations of the labels as y.

### 4. Finding the optimal algorithm to performe sentance sentimental analysis.

We considered specificly these tree models to implement because:
Naive Bayes method was often implemented in sentimen analysis and could either have a very good score or very bad depending on your data type and preprocessing. This model also seemed as one of the closest to the types of models that the book implemented therefore I considered this model to be most relevant.

Logistic Regression model is said to be a very precise model in general with very comprehensible outputs and was said to be easy to implement in general. The book also implements this method so I thought that I will have enough documentation on it to be able to understand it in depth.

Decision Tree model is also a very popular option in sentimental analysis because of it's abilities of generalization. By that I mean that if you find a perfect grouping size for the model to use you can be able to predict sentiments in a very accurate way. 

Continuing:
Now that I have preprocessed data I can use that tougheter with the GridSearchCV() method in order to find the best algorithm and the best parameters for that algorithm to predict the sentiments of the sentances. 

I do that by implementing exhoustive search on the parameters for each of the algorithms.

In [8]:
'''
This function performs a grid search to find the optimal hyperparameter 'var_smoothing' for a Gaussian Naive Bayes Classifier.

Parameters:
x_ax (array-like): The feature matrix (X) of the training data.
y_ax (array-like): The target vector (y) of the training data.

Returns:
float: The optimal value for the hyperparameter 'var_smoothing'.

Function overview:

    Define the hyperparameter 'var_smoothing' to search in a dictionary called 'hypeOmeters'.
    'var_smoothing' will be searched among [1e-9, 1e-8, 1e-7, 1e-6, 1e-5].

    Create a GridSearchCV instance using a GaussianNB model, the defined grid of hyperparameters, and a 5-fold cross-validation.

    Fit the GridSearchCV instance to the training data (x_ax and y_ax).

    Extract the best hyperparameter 'var_smoothing' found during the grid search.

    Return the optimal value for the hyperparameter 'var_smoothing'.
'''

"\nThis function performs a grid search to find the optimal hyperparameter 'var_smoothing' for a Gaussian Naive Bayes Classifier.\n\nParameters:\nx_ax (array-like): The feature matrix (X) of the training data.\ny_ax (array-like): The target vector (y) of the training data.\n\nReturns:\nfloat: The optimal value for the hyperparameter 'var_smoothing'.\n\nFunction overview:\n\n    Define the hyperparameter 'var_smoothing' to search in a dictionary called 'hypeOmeters'.\n    'var_smoothing' will be searched among [1e-9, 1e-8, 1e-7, 1e-6, 1e-5].\n\n    Create a GridSearchCV instance using a GaussianNB model, the defined grid of hyperparameters, and a 5-fold cross-validation.\n\n    Fit the GridSearchCV instance to the training data (x_ax and y_ax).\n\n    Extract the best hyperparameter 'var_smoothing' found during the grid search.\n\n    Return the optimal value for the hyperparameter 'var_smoothing'.\n"

In [9]:
def naiveBayes(x_ax, y_ax):
    hypeOmeters = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}
    
    grid = GridSearchCV(GaussianNB(), hypeOmeters, cv=5)
    grid.fit(x_ax, y_ax)
    
    return grid.best_params_['var_smoothing']

In [10]:
'''
This function performs a grid search to find the optimal hyperparameter 'C' for a Logistic Regression Classifier.

Parameters:
x_ax (array-like): The feature matrix (X) of the training data.
y_ax (array-like): The target vector (y) of the training data.

Returns:
float: The optimal value for the hyperparameter 'C'.

Function overview:

    Define the hyperparameter 'C' to search in a dictionary called 'C_meters'.
    'C' will be searched among [0.001, 0.01, 0.1, 1, 10].

    Create a GridSearchCV instance using a LogisticRegression model, the defined grid of hyperparameters, and a 5-fold cross-validation.

    Fit the GridSearchCV instance to the training data (x_ax and y_ax).

    Extract the best hyperparameter 'C' found during the grid search.

    Return the optimal value for the hyperparameter 'C'.
'''

"\nThis function performs a grid search to find the optimal hyperparameter 'C' for a Logistic Regression Classifier.\n\nParameters:\nx_ax (array-like): The feature matrix (X) of the training data.\ny_ax (array-like): The target vector (y) of the training data.\n\nReturns:\nfloat: The optimal value for the hyperparameter 'C'.\n\nFunction overview:\n\n    Define the hyperparameter 'C' to search in a dictionary called 'C_meters'.\n    'C' will be searched among [0.001, 0.01, 0.1, 1, 10].\n\n    Create a GridSearchCV instance using a LogisticRegression model, the defined grid of hyperparameters, and a 5-fold cross-validation.\n\n    Fit the GridSearchCV instance to the training data (x_ax and y_ax).\n\n    Extract the best hyperparameter 'C' found during the grid search.\n\n    Return the optimal value for the hyperparameter 'C'.\n"

In [11]:
def logisticRegression(x_ax, y_ax):
    C_meters = {'C': [0.001, 0.01, 0.1, 1, 10]}
    
    grid = GridSearchCV(LogisticRegression(), C_meters, cv=5)
    grid.fit(x_ax, y_ax)
    
    return grid.best_params_['C']

In [12]:
'''
This function performs a grid search to find the optimal hyperparameters for a Decision Tree Classifier.

Parameters:
x_ax (array-like): The feature matrix (X) of the training data.
y_ax (array-like): The target vector (y) of the training data.

Returns:
tuple: A tuple containing the optimal values for 'max_depth' and 'min_samples_split'.

Function overview:

    Define the hyperparameters to search in a dictionary called 'grido_meters'.
    'max_depth' will be searched among [2, 4, 6, 8, 10], and 'min_samples_split' among [2, 5, 10, 20].

    Create a GridSearchCV instance using a DecisionTreeClassifier model, the defined grid of hyperparameters, and a 5-fold cross-validation.

    Fit the GridSearchCV instance to the training data (x_ax and y_ax).

    Extract the best hyperparameters ('max_depth' and 'min_samples_split') found during the grid search.

    Return the optimal values for 'max_depth' and 'min_samples_split' as a tuple.
'''

"\nThis function performs a grid search to find the optimal hyperparameters for a Decision Tree Classifier.\n\nParameters:\nx_ax (array-like): The feature matrix (X) of the training data.\ny_ax (array-like): The target vector (y) of the training data.\n\nReturns:\ntuple: A tuple containing the optimal values for 'max_depth' and 'min_samples_split'.\n\nFunction overview:\n\n    Define the hyperparameters to search in a dictionary called 'grido_meters'.\n    'max_depth' will be searched among [2, 4, 6, 8, 10], and 'min_samples_split' among [2, 5, 10, 20].\n\n    Create a GridSearchCV instance using a DecisionTreeClassifier model, the defined grid of hyperparameters, and a 5-fold cross-validation.\n\n    Fit the GridSearchCV instance to the training data (x_ax and y_ax).\n\n    Extract the best hyperparameters ('max_depth' and 'min_samples_split') found during the grid search.\n\n    Return the optimal values for 'max_depth' and 'min_samples_split' as a tuple.\n"

In [13]:
def decissionTree(x_ax, y_ax):
    grido_meters = { 'max_depth': [2, 4, 6, 8, 10], 'min_samples_split': [2, 5, 10, 20] }

    grid = GridSearchCV(DecisionTreeClassifier(), grido_meters, cv=5)
    grid.fit(x_ax, y_ax)
    
    max_depth, min_sample = grid.best_params_['max_depth'], grid.best_params_['min_samples_split']
    return max_depth, min_sample


And here I will create a function that will test the different algorithm with their best parameters and present the best algorithm with it's best parameter. 

In [14]:
def optimal_paramteres(sentances, sentiments, y_axis, densed_X):
    Logistic_param = logisticRegression(
    x_ax = densed_X, 
    y_ax = y_axis
    )

    Naive_param = naiveBayes(
        x_ax = densed_X, 
        y_ax = y_axis
    )

    DecTree_depth_param, DecTree_sample_param = decissionTree(
        x_ax = densed_X, 
        y_ax = y_axis
    )

    return [Logistic_param, Naive_param, DecTree_depth_param, DecTree_sample_param]

In [15]:
sentances , sentiments = preprocessing(train_data)

x_ax, y_ax, vectorizer, labelizer = vectornihilation(sentances, sentiments)

densed_X = x_ax.toarray()

optimal_paramteres = optimal_paramteres(
    sentances = sentances,
    sentiments = sentiments,
    y_axis = y_ax,
    densed_X = densed_X
)

logistic_param, naive_param, decTree_depth_param, decTree_sample_param = optimal_paramteres

In [26]:
def OptimalSentimentAnalysis(densed_X, y_axis, model, param1, param2=None):
    if param2 is None:
        if model == GaussianNB:
            Cross_Validation = cross_val_score(
                model(var_smoothing=param1),
                densed_X,
                y_axis
            )
        elif model == LogisticRegression:
            Cross_Validation = cross_val_score(
                model(C=param1),
                densed_X,
                y_axis
            )
    else:
        if model == DecisionTreeClassifier:
            Cross_Validation = cross_val_score(
                model(max_depth=param1, min_samples_split=param2),
                densed_X,
                y_axis
            )
    return Cross_Validation


    Cross_Naive_score = cross_val_score(
        GaussianNB(var_smoothing = Naive_param), 
        densed_X, 
        y_axis)
    
    Cross_Logistic_score = cross_val_score(
        LogisticRegression(C = Logistic_param), 
        densed_X, 
        y_axis
        )
    
    Cross_DecTree_score = cross_val_score(
        DecisionTreeClassifier(
            max_depth = DecTree_depth_param, 
            min_samples_split = DecTree_sample_param
            ), 
        densed_X, 
        y_axis
        )
    
    printCrossInfo(Cross_Naive_score)
    printCrossInfo(Cross_Logistic_score)
    printCrossInfo(Cross_DecTree_score)

    return Cross_Naive_score, Cross_Logistic_score, Cross_DecTree_score
    # print(f'Naive Bayes algorithm scored: {Cross_Naive_score.mean()}')
    # print(f'LogisticRegression algorithm scored: {Cross_Logistic_score.mean()}')
    # print(f'Decission Tree scored: {Cross_DecTree_score.mean()}')

def printCrossInfo(model):
    print(f'Model: {type(model).__name__} | Cross-validation score: {model.mean()}')
    print(f'Model: {type(model).__name__} | Cross-validation standard deviation: {model.std()}')

In [27]:
a = OptimalSentimentAnalysis(
    densed_X = densed_X,
    y_axis = y_ax,
    model = GaussianNB,
    param1 = naive_param
)

b = OptimalSentimentAnalysis(
    densed_X = densed_X,
    y_axis = y_ax,
    model = LogisticRegression,
    param1 = logistic_param
)

c = OptimalSentimentAnalysis(
    densed_X = densed_X,
    y_axis = y_ax,
    model = DecisionTreeClassifier,
    param1 = decTree_depth_param,
    param2 = decTree_sample_param
)

In [28]:
printCrossInfo(a)
printCrossInfo(b)
printCrossInfo(c)

Model: ndarray | Cross-validation score: 0.582878949118723
Model: ndarray | Cross-validation standard deviation: 0.010137512927733272
Model: ndarray | Cross-validation score: 0.7079115397406053
Model: ndarray | Cross-validation standard deviation: 0.00815768980931966
Model: ndarray | Cross-validation score: 0.6911543066178917
Model: ndarray | Cross-validation standard deviation: 0.007182422326052021


Here I quickly explain what different parameters does in the different models:
var_smoothing variable chooses how smoothly the variance should be distributed in the Naive Bayes distribution model.

C is a value that regularates the complexity of the Logistic Regregression model. What I try to find is the maximum Likelyhood parameter. Which is the parameter that gives me the optimal values for the likelyhood of getting Y given an X.

max_depth is the maximum depth of the decision tree. A decision tree can become very complex and can overfit the training data if it is allowed to grow too deep. So, setting a maximum depth for the tree can help to prevent overfitting and improve generalization to new data.

min_samples_split chooses how small the decision groups can be, preventing this parameter from being to small can help in preventing overfitting by creating a more general model that fits more data. 



And now I that I have found that Logistic Regression is the model that I am supposed to use I can validate the accuracy of the model before testing it:

In [ ]:
def LogisticSentimentAnalysisRegression_trainANDValidate(training_x, training_y, validating_x, validating_y, module):
    # The evaluation:
    module.fit(training_x, training_y)
    
    validity_responses = module.predict(validating_x)
    
    accuracy = accuracy_score(validating_y, validity_responses)
    report = classification_report(validating_y, validity_responses)

    print(f"{module}:\nAccuracy: {accuracy}\n{report}\n")


In [ ]:
def performance():
    LogiReg = LogisticRegression()  # Instantiating the model.

    train_sentances, train_sentiments = preprocessing(data_reading(json_training_data))

    sentance_vectors_x, sentiment_vectors_y, vectorizer, labelizer = vectornihilation(train_sentances, train_sentiments)
    fixed_SV_x = sentance_vectors_x.toarray()  # Convert the matrix to dense array to process it

    valid_sentances, valid_sentiments = preprocessing(data_reading(json_test_data))
    
    # Use the same vectorizer and labelizer for transforming validation data
    validSentancesVectors_x = vectorizer.transform(valid_sentances)
    validSentimentVectors_Y = labelizer.transform(valid_sentiments)
    
    fixed_ValidSV_x = validSentancesVectors_x.toarray()

    LogisticSentimentAnalysisRegression_trainANDValidate(fixed_SV_x, sentiment_vectors_y, fixed_ValidSV_x, validSentimentVectors_Y, LogiReg)

performance()